In [ ]:
import mlrun
project_name = "test-serv-with-remote"
# get/create a project and register the data prep and trainer function in it
project = mlrun.get_or_create_project(
    name=project_name, user_project=False, context="./"
)

<a id="serving-function"></a>
## Create and test the Serving Function

The serving function kind is using [Nuclio](https://nuclio.io/): a high-performance serverless event 
and data processing open-source platform. With just a few lines of code we can take our model, **expose** it with an **http endpoint** and deploy it on 
**high-performance** infrastructure that can easily scale up to serve on a **production environment** with hundreds of 
thousands of requests per second. To read more about serving functions in MLRun and some cool advanced features like model routers, error handling and more, please refer to our [documentation](https://docs.mlrun.org/en/latest/serving/serving-graph.html).


In [ ]:
from mlrun.frameworks.lgbm import LGBMModelServer
remote_func_name = "predict_remote"

fn_remote = mlrun.new_function(remote_func_name,
                               kind="serving", 
                               image="mlrun/mlrun", requirements=["lightgbm"])


fn_remote.add_model("lgbm_ny_taxi", class_name=LGBMModelServer(None, name="lgbm_ny_taxi", model_path=project.get_artifact('lgbm_ny_taxi').uri))
remote_addr = fn_remote.deploy()

In [ ]:
serving_function = project.set_function(name='serving', func='src/serving.py', image='mlrun/mlrun', kind="serving", requirements=["lightgbm"])

In [ ]:
# Set the topology and get the graph object:
graph = serving_function.set_topology("flow", engine="async")

In [ ]:
from mlrun.feature_store.steps import DateExtractor

# Build the serving graph:
graph.to(handler="add_airport_dist", name="calculate_airport_distance")\
     .to(handler="radian_conv_step", name="calculate_radian_conv")\
     .to(handler="sphere_dist_bear_step", name="bearing_calculation")\
     .to(handler="sphere_dist_step", name="distance_calculation")\
     .to(DateExtractor(parts=["hour", "day", 'month', "day_of_week", 'year'],timestamp_col="pickup_datetime"))\
     .to(handler="preprocess", name="preprocess")\
     .to("$remote", remote_func_name, url=f'{remote_addr}v2/models/lgbm_ny_taxi/infer', method='put')\
     .to(handler="postprocess", name="postprocess").respond()

# Plot to graph:
graph.plot(rankdir='LR')

In [ ]:
# create a mock server (in memory simulator for the graph for testing)
server = serving_function.to_mock_server()

body = {'pickup_longitude':-73.844311, 'pickup_latitude':40.721319,
        'dropoff_longitude':-73.84161, 'dropoff_latitude': 40.712278,
        'passenger_count':1,
        'pickup_datetime': '2013-01-01T12', 'key': '2013-01-01T12'}

# simulate a user request and print the results
response_mock = server.test(path="/v2/models/lgbm_ny_taxi/infer", body=body.copy())
print(response_mock['result_str'])

<a id="deploy-serving"></a>
## Deploy the serving Function


In [ ]:
# Deploy it:
deployment = project.deploy_function(serving_function)

In [ ]:
response = deployment.function.invoke(path='/v2/models/lgbm_ny_taxi/infer', body=body.copy())
print(response)

In [ ]:
assert response['result_str'] == response_mock['result_str']